In [ ]:
# test UDS Core

import os
from pathlib import Path
from openai import OpenAI

def read_file(file_path):
    """Read the contents of a file."""
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def process_markdown_files(folder_path):
    """Process all .md and .mdx files in the given folder and its subfolders."""
    combined_content = ""
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith(('.md', '.mdx')):
                file_path = Path(root) / file
                content = read_file(file_path)
                combined_content += f"\n\nContent of {file_path}:\n{content}"
    return combined_content

def query_llm(logs, events, docs, query, model="gpt-4"):
    """Query the LLM about the K8s cluster state with additional context."""
    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are an expert in Kubernetes cluster analysis and the air-gap software delivery tool Zarf. Docs for Zarf as are follows:\n\n{docs} \n\nAnalyze the provided logs and events to answer questions about the cluster's current state."},
            {"role": "user", "content": f"Here are the recent logs from a Kubernetes cluster:\n\n{logs}\n\nHere are the recent events:\n\n{events}\n\nBased on all this information, please answer the following question: {query}\n\n Be brief in your response, no need for any in-depth analysis"}
        ]
    )
    return response.choices[0].message.content

log_file = "logs.txt"
event_file = "events.txt"
docs_folder = "../zarf/site/src/content/docs"
model = "gpt-4o-mini"
    
# logs = read_file(log_file)
# events = read_file(event_file)
# docs_content = process_markdown_files(docs_folder)

query = """
    Give me a general overview of the state and health of my cluster
"""

answer = query_llm(logs, events, docs_content, query, model)
print("\nAnalysis:")
print(answer)